In [1]:
import math
import collections
import pickle
import random
import scipy
import json

from matplotlib import pyplot as plt

import GPy
import numpy as np
import pandas as pd
from tqdm import tqdm


### 1. Perform conversions from volumes and carbon source name to design space

In [2]:
Stock_solid = pd.read_excel('../CarbonSourceInfo.xlsx', 'Stocks_solid')
Stock_liquid = pd.read_excel('../CarbonSourceInfo.xlsx', 'Stocks_liquid')

CARBON NAME

In [3]:
Carbon_Names = Stock_solid['Carbon Source'].values.tolist()
Carbon_Names.append(Stock_liquid['Carbon Source'][1])
Carbon_Names.append(Stock_liquid['Carbon Source'][2])

UPPER BOUNDS

In [4]:
Carbon_Ub = [50]*17 #[g/L]
Carbon_Ub.append(10) # Glycerol [mL/L]
Carbon_Ub.append(50) #Ethanol [mL/L]

OG_Gly_Ub = 100 #[mL/L]
Met_Ub = 100 #[mL/L]

STOCK CONCENTRATIONS

In [5]:
Stock_Conc = Stock_solid['g/mL'].values.tolist()
Stock_Conc.append(Stock_liquid['mL/mL'][1].tolist())
Stock_Conc.append(Stock_liquid['mL/mL'][2].tolist())

OG_Stock_Conc = Stock_liquid['mL/mL'][0].tolist()

In [6]:
Exp_Used = pd.read_csv('Actual_Round0_3mL.csv')
initN = Exp_Used.shape[0]
print(initN)
Exp_Used_CT = Exp_Used["Carbon_Type"].values
Exp_Used_CC = Exp_Used["Conc_Carbon [uL]"].values
Exp_Used_Gly = Exp_Used["Gly_OG [uL]"].values
Exp_Used_Met = Exp_Used["Met_Prod [uL]"].values

23


In [7]:
Des_recon_CT = []
Des_recon_CC = []
Des_recon_Gly = []
Des_recon_Met = []

for i in range(initN):
    index = Carbon_Names.index(Exp_Used_CT[i])
    Des_recon_CT.append(index)
    temp_factor = Carbon_Ub[index]/ Stock_Conc[index]
    cc = Exp_Used_CC[i]/ (temp_factor * 3)
    Des_recon_CC.append(cc)
    gly = Exp_Used_Gly[i] * OG_Stock_Conc/(OG_Gly_Ub * 3)
    Des_recon_Gly.append(gly)
    met = Exp_Used_Met[i]/( Met_Ub * 3)
    Des_recon_Met.append(met)

In [8]:
Design_actual = {'Carbon_Type': Des_recon_CT,
               'Conc_Carbon': Des_recon_CC,
               'Gly_OG': Des_recon_Gly,
               'Met_Prod': Des_recon_Met}

In [9]:
pd.DataFrame(Design_actual).to_csv('Reconstructed_Round0.csv')

In [10]:
Carbon_Names

['Sorbitol',
 'Mannitol',
 'Xylitol',
 'Glucose',
 'Mannose',
 'Pyruvate',
 'Fructose',
 'Xylose',
 'Galactose',
 'D-Arabinose',
 'Rhamnose',
 'Sucrose',
 'Lactose',
 'Trehalose',
 'Ribose',
 'Citrate',
 'Succinate',
 'Glycerol ',
 'Ethanol']